# gp_cross_factor 功能展示

In [1]:
### 导入相关内容
import warnings
import os
import numpy as np
import pandas as pd
from add_ts_function import ts_std_10, ts_max_10,  ts_mean_10
import genetic
np.random.seed(10)
pd.set_option('display.max_columns', None)
pd.set_option('expand_frame_repr', True)
pd.set_option('display.unicode.ambiguous_as_wide', True)
warnings.filterwarnings('ignore')


In [2]:
Y = np.load("./Y.npy")
print(Y.shape)

(728, 4984)


In [3]:
X = np.load("./X.npy")
print(X.shape)

(728, 6, 4984)


In [ ]:
feature_names = ["open", "close", "high", "low","vwap","volume"]

In [6]:
max_samples=0.8
sample_weight = np.ones(X.shape[0])
num_div = int(X.shape[0] * max_samples)
sample_weight[num_div:] = 0

In [7]:
function_set_sample = ['common_add', 'common_sub', 'common_mul', 'common_div',
                       'common_log', 'common_sqrt', 'common_abs', 'common_inv', 'common_max', 'common_min', 'common_tan'] # ,'common_sin','common_cos','common_neg'] 
my_function = [ts_std_10, ts_max_10,  ts_mean_10,]
function_set = function_set_sample + my_function

In [8]:
gp_sample = genetic.SymbolicTransformer(generations=4,
                                        population_size=200,
                                        tournament_size=20,
                                        init_depth=(1,4),
                                        hall_of_fame=100,
                                        n_components=20,
                                        function_set=function_set,
                                        metric="ic",
                                        const_range=(-1, 1),
                                        p_crossover=0.4,
                                        p_hoist_mutation=0.001,
                                        p_subtree_mutation=0.01,
                                        p_point_mutation=0.01,
                                        p_point_replace=0.4,
                                        parsimony_coefficient="auto",
                                        feature_names=feature_names,
                                        max_samples=max_samples, verbose=1,
                                        random_state=0, n_jobs=-4)

In [ ]:
gp_sample.fit_3D(X, Y,feature_names,sample_weight=sample_weight,baseIC=0.02,need_parallel=True)

In [10]:
baseIC_best_result = gp_sample.show_program(X,Y,feature_names,baseIC=True)
res = pd.DataFrame(baseIC_best_result).drop_duplicates(subset="表达式").sort_values(by='fitness',ascending = False)
res.to_csv("./GP_demo.csv",index=False)

In [11]:
res

,表达式,fitness,OOB fitness
393,"ts_max_10(common_min(common_div(close, low), c...",0.048261,0.057060
89,ts_max_10(common_min(common_sub(common_div(vwa...,0.048100,0.056972
238,ts_max_10(common_min(common_sub(common_div(hig...,0.047534,0.056391
57,ts_max_10(common_min(common_sub(common_div(hig...,0.047353,0.056167
182,ts_max_10(common_min(common_sub(common_div(hig...,0.047353,0.056167
...,...,...,...
28,"common_min(ts_std_10(low), common_sub(volume, ...",0.022198,0.027144
32,"ts_mean_10(ts_std_10(common_min(open, low)))",0.021787,0.024736
3,ts_std_10(common_mul(common_min(common_sub(vwa...,0.021091,0.024621
31,"common_sub(ts_max_10(common_mul(-0.298, volume...",0.021028,0.028567
